In [1]:
import json
import os
import pandas as pd
import numpy as np
import math
from IPython.display import display

In [2]:
correct_answers = {
    "nl_t2_clique_tl" : 3,
    "m_t2_clique_jp" : 3,
    "m_t2_single_si" : 2,
    "m_t1_single_si" : 2,
    "nl_t1_clique_jp" : 4,
    "nl_t2_single_tl" : 4,
    "m_t1_clique_si" : 3,
    "nl_t1_clique_tl" : 1,
    "nl_t3_clique_an" : 4,
    "m_t1_clique_tl" : 1,
    "m_t1_clique_an" : 2,
    "nl_t3_clique_jp" : 3,
    "m_t1_single_jp" : 3,
    "nl_t1_single_jp" : 2,
    "m_t2_single_tl" : 4,
    "nl_t2_single_an" : 4,
    "nl_t1_single_si" : 3,
    "m_t3_single_si" : 3,
    "m_t3_single_an" : 4,
    "nl_t3_clique_si" : 5,
    "m_t2_clique_an" : 4,
    "nl_t1_single_tl" : 3,
    "m_t3_single_tl" : 4,
    "m_t2_clique_si" : 6,
    "m_t2_single_jp" : 4,
    "nl_t3_single_jp" : 1,
    "nl_t1_clique_si" : 3,
    "m_t1_single_an" : 2,
    "nl_t2_single_si" : 3,
    "nl_t3_single_si" : 1,
    "m_t3_clique_si" : 5,
    "m_t1_single_tl" : 3,
    "m_t3_clique_tl" : 1,
    "nl_t2_single_jp" : 4,
    "nl_t3_clique_tl" : 1,
    "nl_t3_single_tl" : 2,
    "m_t2_single_an" : 2,
    "m_t3_clique_an" : 4,
    "nl_t2_clique_jp" : 3,
    "m_t3_clique_jp" : 3,
    "nl_t1_clique_an" : 2,
    "nl_t3_single_an" : 2,
    "nl_t1_single_an" : 4,
    "nl_t2_clique_si" : 6,
    "nl_t2_clique_an" : 4,
    "m_t3_single_jp" : 1,
    "m_t2_clique_tl" : 3,
    "m_t1_clique_jp" : 4,
    "control1": "red",
    "control2": 4,
    "control3": "LT02"
}

In [3]:
def is_correct(answer, question):
    return answer == correct_answers[question]

In [4]:
def sum_timers(timers): 
    sums = {}
    if type(timers) is list: 
        for i in timers:
            if i['type'] not in sums:
                sums[i['type']] = i['time']
            else:
                sums[i['type']] = sums[i['type']] + i['time'] # Return as list instead of sum,
            # TODO: Convert from ms to s?
    return sums

In [5]:
def list_timers(timers): 
    lists = {
        'zoom': [],
        'drag': [],
        'highlight': [],
        'slower': [],
        'faster': [],
        'slider': []
    }
    if type(timers) is list: 
        for i in timers:
            lists[i['type']].append(i['time'])
            # TODO: Convert from ms to s?
    return lists

In [6]:
def parse_result(df, group, result): 
    # group, result_id, question_id, answer, confidence, time, interaaction_timer, type_interaction, correct, diff
    data = []
    
    question = ''
    answer = 0
    time = 0
    confidence = 0
    correct = False
    
    zoom_timers = []
    drag_timers = []
    highlight_timers = []
    slower_timers = []
    faster_timers = []
    slider_timers = []
    
    zoom_interactions = 0
    drag_interactions = 0
    highlight_interactions = 0
    slower_interactions = 0
    faster_interactions = 0
    slider_interactions = 0
    
    for index, row in df.iterrows():
        survey_question = row['question']
     
        if 'preferences' in survey_question: #Preferences
#             print('preferences')
            continue
        elif 'comments' in survey_question: # Comments
#             print('comments')
            continue
        elif 'confidence' not in survey_question[0]: # Question Answer   
            # Separate for NL and M
            question = row['question'][0]
            answer = row['question'][1]
            time = row['time']
            
            interactions = row['interactions']
            
            timers = row['timers']
            lists = list_timers(row['timers'])
        
        if (index+1)%2 == 0:
            if 'preferences' not in survey_question and 'comments' not in survey_question:
                confidence = row['question'][1]
                correct = is_correct(answer, question)
                
                # Arrays of timers for each interaction
                zoom_timers = lists['zoom']
                drag_timers = lists['drag']
                highlight_timers = lists['highlight']
                slower_timers = lists['slower']
                faster_timers = lists['faster']
                slider_timers = lists['slider']
                
                # Number of total interactions per interaction type
                if type(interactions) is dict: 
                    zoom_interactions = interactions.get('zooms')
                    drag_interactions = interactions.get('drags')
                    highlight_interactions = interactions.get('highlights')
                    slower_interactions = interactions.get('slower')
                    faster_interactions = interactions.get('faster')
                    slider_interactions = interactions.get('slider')
                
                data.append([group, result, question, answer, time, confidence, correct_answers.get(question), correct,
                             zoom_timers, drag_timers, highlight_timers, slower_timers, faster_timers, slider_timers,
                             zoom_interactions, drag_interactions, highlight_interactions, slower_interactions, faster_interactions, slider_interactions])

                question = ''
                answer = 0
                time = 0
                confidence = 0
                correct = False
                
                zoom_timers = []
                drag_timers = []
                highlight_timers = []
                slower_timers = []
                faster_timers = []
                slider_timers = []

                zoom_interactions = 0
                drag_interactions = 0
                highlight_interactions = 0
                slower_interactions = 0
                faster_interactions = 0
                slider_interactions = 0
            
    results = pd.DataFrame(data, columns = ['group', 'id', 'question', 'answer', 'time', 'confidence', 'correct_answer', 'correct', 'zoom_timers', 'drag_timers', 'highlight_timers', 'slower_timers', 'faster_timers', 'slider_timers', 'zoom_interactions', 'drag_interactions', 'highlight_interactions', 'slower_interactions', 'faster_interactions', 'slider_interactions'])
    return results

In [7]:
def parse_feedback(df, group, result):
    # group, result_id, NL-JP, NL-SI, NL-TL, NL-AN, M-JP, M-SI, M-TL, M-AN, comments
    data = []
    
    nl_jp = 0
    nl_si = 0
    nl_tl = 0
    nl_an = 0
    m_jp = 0
    m_si = 0
    m_tl = 0
    m_an = 0
    comments = ""
    
    for index, row in df.iterrows():
        survey_question = row['question']
        if 'preferences' in survey_question: #Preferences
            
            nl_jp = survey_question.get('preferences').get('NL').get('JP')
            nl_si = survey_question.get('preferences').get('NL').get('SI')
            nl_tl = survey_question.get('preferences').get('NL').get('TL')
            nl_an = survey_question.get('preferences').get('NL').get('AN')
            
            m_jp = survey_question.get('preferences').get('M').get('JP')
            m_si = survey_question.get('preferences').get('M').get('SI')
            m_tl = survey_question.get('preferences').get('M').get('TL')
            m_an = survey_question.get('preferences').get('M').get('AN')

        elif 'comments' in survey_question: # Comments
            
            comments = survey_question.get('comments')
            
        if index == len(df)-1:           
            data.append([group, result, nl_jp, nl_si, nl_tl, nl_an, m_jp, m_si, m_tl, m_an, comments])
            
            nl_jp = 0
            nl_si = 0
            nl_tl = 0
            nl_an = 0
            m_jp = 0
            m_si = 0
            m_tl = 0
            m_an = 0
            comments = ""
    
    results = pd.DataFrame(data, columns = ['group', 'id', 'NL-JP', 'NL-SI', 'NL-TL', 'NL-AN', 'M-JP', 'M-SI', 'M-TL', 'M-AN', 'comments'])
    return results

In [8]:
# Go through results dir and load all files in data frame
def read_files(): 
    directory = r'results'
    survey_count = 0
    survey_results = pd.DataFrame([], columns = ['group', 'id', 'question', 'answer', 'time', 'confidence', 'correct_answer', 'correct', 'zoom_timers', 'drag_timers', 'highlight_timers', 'slower_timers', 'faster_timers', 'slider_timers', 'zoom_interactions', 'drag_interactions', 'highlight_interactions', 'slower_interactions', 'faster_interactions', 'slider_interactions'])
    feedback_results = pd.DataFrame([], columns = ['group', 'id', 'NL-JP', 'NL-SI', 'NL-TL', 'NL-AN', 'M-JP', 'M-SI', 'M-TL', 'M-AN', 'comments'])
    for filename in os.listdir(directory):
    
        if filename.endswith(".json"):
            with open(directory + '/' + filename, encoding='utf-8', mode='r') as current_file:
                
                text = os.path.splitext(filename)[0]
                # parse id and group from file name
                result_id = text.split('_')[1]
                group = text.split('_')[2]
                
                df = pd.read_json(current_file)
                
                data = parse_result(df, group, result_id) # parse single survey result
                feedback = parse_feedback(df, group, result_id) # parse feedback
                
                survey_results = survey_results.append(data, ignore_index = True) # append to results
                feedback_results = feedback_results.append(feedback, ignore_index = True) # append to results
                survey_count+=1
        else:
            print('File not JSON', filename)
            continue
    survey_results = survey_results.fillna(0)
    survey_results = survey_results.astype({
        'zoom_interactions': int,
        'drag_interactions': int,
        'highlight_interactions': int,
        'slower_interactions': int,
        'faster_interactions': int,
        'slider_interactions': int,
        'time': int,
    })
#     survey_results['drag_interactions'].fillna(0)
#     survey_results['highlight_interactions'].fillna(0)
#     survey_results['slower_interactions'].fillna(0)
#     survey_results['faster_interactions'].fillna(0)
#     survey_results['slider_interactions'].fillna(0)
    
    print(f'Parsed {survey_count} surveys.')
#     display(survey_results)
    return [survey_results, feedback_results]

In [9]:
def save_csv(results, name): 
    results.to_csv(f'{name}.csv')
    print(f'CSV generated \'{name}.csv\'')

In [10]:
survey_data, feedback_data = read_files()
save_csv(survey_data, 'survey_results')
save_csv(feedback_data, 'feedback_results')

0 ['m_t1_clique_si', 3]
1 ['m_t1_clique_si_confidence', 1]
2 ['nl_t3_single_si', 1]
3 ['nl_t3_single_si_confidence', 5]
4 ['m_t1_single_tl', 3]
5 ['m_t1_single_tl_confidence', 5]
6 ['m_t3_clique_jp', 5]
7 ['m_t3_clique_jp_confidence', 5]
8 ['m_t1_single_si', 2]
9 ['m_t1_single_si_confidence', 5]
10 ['m_t3_clique_tl', 4]
11 ['m_t3_clique_tl_confidence', 4]
12 ['m_t1_single_jp', 3]
13 ['m_t1_single_jp_confidence', 5]
14 ['nl_t1_single_si', 3]
15 ['nl_t1_single_si_confidence', 5]
16 ['nl_t1_single_an', 4]
17 ['nl_t1_single_an_confidence', 5]
18 ['nl_t3_clique_jp', 5]
19 ['nl_t3_clique_jp_confidence', 4]
20 ['nl_t3_clique_jp', 5]
21 ['nl_t3_clique_jp_confidence', 4]
22 ['control1', 'red']
23 ['control1_confidence', 2]
24 ['m_t2_single_si', 4]
25 ['m_t2_single_si_confidence', 4]
26 ['m_t1_clique_tl', 1]
27 ['m_t1_clique_tl_confidence', 5]
28 ['nl_t3_clique_tl', 4]
29 ['nl_t3_clique_tl_confidence', 4]
30 ['m_t2_single_tl', 4]
31 ['m_t2_single_tl_confidence', 4]
32 ['m_t2_clique_si', 5]
33 ['

37 ['m_t2_clique_tl_confidence', 2]
38 ['nl_t1_single_an', 4]
39 ['nl_t1_single_an_confidence', 5]
40 ['m_t1_clique_tl', 1]
41 ['m_t1_clique_tl_confidence', 3]
42 ['m_t3_clique_an', 4]
43 ['m_t3_clique_an_confidence', 2]
44 ['nl_t3_single_an', 2]
45 ['nl_t3_single_an_confidence', 5]
46 ['nl_t3_clique_si', 5]
47 ['nl_t3_clique_si_confidence', 3]
48 ['nl_t3_clique_an', 4]
49 ['nl_t3_clique_an_confidence', 3]
50 ['m_t3_clique_jp', 5]
51 ['m_t3_clique_jp_confidence', 3]
52 ['m_t3_clique_jp', 5]
53 ['m_t3_clique_jp_confidence', 3]
54 ['control2', 4]
55 ['control2_confidence', 5]
56 ['nl_t2_clique_tl', 2]
57 ['nl_t2_clique_tl_confidence', 4]
58 ['nl_t1_clique_tl', 1]
59 ['nl_t1_clique_tl_confidence', 4]
60 ['m_t1_clique_si', 3]
61 ['m_t1_clique_si_confidence', 4]
62 ['m_t2_single_si', 2]
63 ['m_t2_single_si_confidence', 3]
64 ['m_t2_clique_jp', 3]
65 ['m_t2_clique_jp_confidence', 3]
66 ['m_t2_clique_si', 6]
67 ['m_t2_clique_si_confidence', 2]
68 ['m_t3_single_si', 3]
69 ['m_t3_single_si_conf

100 ['nl_t3_clique_tl', 4]
101 ['nl_t3_clique_tl_confidence', 4]
102 ['nl_t1_clique_jp', 2]
103 ['nl_t1_clique_jp_confidence', 5]
104 {'preferences': {'NL': {'JP': 4, 'SI': 3, 'AN': 5, 'TL': 4}, 'M': {'JP': 4, 'SI': 3, 'AN': 5, 'TL': 4}}}
105 {'comments': '-'}
0 ['m_t2_clique_an', 3]
1 ['m_t2_clique_an_confidence', 4]
2 ['m_t3_single_jp', 1]
3 ['m_t3_single_jp_confidence', 5]
4 ['m_t3_clique_an', 4]
5 ['m_t3_clique_an_confidence', 3]
6 ['m_t1_single_si', 2]
7 ['m_t1_single_si_confidence', 4]
8 ['m_t1_single_tl', 3]
9 ['m_t1_single_tl_confidence', 5]
10 ['m_t1_single_an', 2]
11 ['m_t1_single_an_confidence', 5]
12 ['nl_t1_clique_jp', 4]
13 ['nl_t1_clique_jp_confidence', 5]
14 ['nl_t3_single_an', 2]
15 ['nl_t3_single_an_confidence', 5]
16 ['nl_t1_clique_tl', 1]
17 ['nl_t1_clique_tl_confidence', 5]
18 ['m_t2_clique_si', 6]
19 ['m_t2_clique_si_confidence', 4]
20 ['nl_t3_single_tl', 2]
21 ['nl_t3_single_tl_confidence', 5]
22 ['nl_t3_single_tl', 2]
23 ['nl_t3_single_tl_confidence', 5]
24 ['co

0 ['nl_t2_clique_si', 4]
1 ['nl_t2_clique_si_confidence', 2]
2 ['m_t3_single_tl', 4]
3 ['m_t3_single_tl_confidence', 5]
4 ['nl_t3_single_an', 2]
5 ['nl_t3_single_an_confidence', 5]
6 ['m_t1_single_an', 2]
7 ['m_t1_single_an_confidence', 4]
8 ['nl_t2_clique_jp', 6]
9 ['nl_t2_clique_jp_confidence', 2]
10 ['nl_t3_single_si', 1]
11 ['nl_t3_single_si_confidence', 4]
12 ['nl_t1_clique_an', 2]
13 ['nl_t1_clique_an_confidence', 5]
14 ['m_t2_clique_si', 5]
15 ['m_t2_clique_si_confidence', 3]
16 ['m_t2_single_jp', 4]
17 ['m_t2_single_jp_confidence', 4]
18 ['nl_t2_clique_tl', 3]
19 ['nl_t2_clique_tl_confidence', 4]
20 ['nl_t3_single_jp', 1]
21 ['nl_t3_single_jp_confidence', 5]
22 ['nl_t3_single_jp', 1]
23 ['nl_t3_single_jp_confidence', 5]
24 ['control1', 'red']
25 ['control1_confidence', 5]
26 ['m_t2_single_si', 3]
27 ['m_t2_single_si_confidence', 4]
28 ['m_t3_clique_tl', 4]
29 ['m_t3_clique_tl_confidence', 4]
30 ['m_t2_clique_an', 3]
31 ['m_t2_clique_an_confidence', 2]
32 ['m_t1_single_jp', 3]
3

51 ['m_t1_single_an_confidence', 5]
52 ['m_t1_single_an', 1]
53 ['m_t1_single_an_confidence', 5]
54 ['control2', 4]
55 ['control2_confidence', 5]
56 ['m_t3_clique_si', 5]
57 ['m_t3_clique_si_confidence', 4]
58 ['nl_t1_single_tl', 1]
59 ['nl_t1_single_tl_confidence', 5]
60 ['nl_t3_clique_si', 4]
61 ['nl_t3_clique_si_confidence', 4]
62 ['nl_t2_clique_tl', 5]
63 ['nl_t2_clique_tl_confidence', 3]
64 ['nl_t3_clique_jp', 4]
65 ['nl_t3_clique_jp_confidence', 3]
66 ['m_t3_single_tl', 4]
67 ['m_t3_single_tl_confidence', 5]
68 ['m_t1_single_si', 2]
69 ['m_t1_single_si_confidence', 5]
70 ['m_t1_clique_tl', 3]
71 ['m_t1_clique_tl_confidence', 3]
72 ['nl_t1_single_si', 3]
73 ['nl_t1_single_si_confidence', 5]
74 ['nl_t1_clique_si', 3]
75 ['nl_t1_clique_si_confidence', 5]
76 ['nl_t2_single_an', 1]
77 ['nl_t2_single_an_confidence', 5]
78 ['nl_t3_single_jp', 1]
79 ['nl_t3_single_jp_confidence', 5]
80 ['nl_t3_single_jp', 1]
81 ['nl_t3_single_jp_confidence', 5]
82 ['control3', 'LT02']
83 ['control3_confi

...

8 ['m_t2_single_si', 3]
9 ['m_t2_single_si_confidence', 4]
10 ['nl_t3_clique_si', 5]
11 ['nl_t3_clique_si_confidence', 5]
12 ['nl_t1_single_si', 3]
13 ['nl_t1_single_si_confidence', 5]
14 ['nl_t2_clique_jp', 3]
15 ['nl_t2_clique_jp_confidence', 4]
16 ['m_t1_single_si', 2]
17 ['m_t1_single_si_confidence', 5]
18 ['m_t3_single_an', 4]
19 ['m_t3_single_an_confidence', 5]
20 ['m_t3_single_si', 3]
21 ['m_t3_single_si_confidence', 5]
22 ['nl_t3_single_an', 2]
23 ['nl_t3_single_an_confidence', 5]
24 ['nl_t3_single_an', 2]
25 ['nl_t3_single_an_confidence', 5]
26 ['control1', 'red']
27 ['control1_confidence', 5]
28 ['nl_t1_clique_si', 1]
29 ['nl_t1_clique_si_confidence', 2]
30 ['nl_t2_clique_si', 6]
31 ['nl_t2_clique_si_confidence', 4]
32 ['nl_t1_single_jp', 2]
33 ['nl_t1_single_jp_confidence', 5]
34 ['m_t3_single_tl', 4]
35 ['m_t3_single_tl_confidence', 5]
36 ['m_t2_single_jp', 4]
37 ['m_t2_single_jp_confidence', 5]
38 ['m_t3_clique_jp', 4]
39 ['m_t3_clique_jp_confidence', 5]
40 ['m_t2_clique_s

59 ['nl_t3_single_si_confidence', 5]
60 ['nl_t1_single_an', 4]
61 ['nl_t1_single_an_confidence', 5]
62 ['m_t3_single_jp', 1]
63 ['m_t3_single_jp_confidence', 5]
64 ['nl_t1_clique_tl', 1]
65 ['nl_t1_clique_tl_confidence', 4]
66 ['nl_t3_single_an', 2]
67 ['nl_t3_single_an_confidence', 5]
68 ['nl_t2_single_jp', 4]
69 ['nl_t2_single_jp_confidence', 5]
70 ['m_t3_clique_jp', 5]
71 ['m_t3_clique_jp_confidence', 5]
72 ['nl_t2_clique_an', 6]
73 ['nl_t2_clique_an_confidence', 4]
74 ['m_t2_clique_si', 6]
75 ['m_t2_clique_si_confidence', 5]
76 ['m_t1_single_si', 2]
77 ['m_t1_single_si_confidence', 5]
78 ['m_t3_single_tl', 4]
79 ['m_t3_single_tl_confidence', 5]
80 ['m_t3_single_tl', 4]
81 ['m_t3_single_tl_confidence', 5]
82 ['control3', 'LT02']
83 ['control3_confidence', 4]
84 ['nl_t2_clique_si', 4]
85 ['nl_t2_clique_si_confidence', 4]
86 ['nl_t3_single_tl', 2]
87 ['nl_t3_single_tl_confidence', 5]
88 ['nl_t1_clique_si', 1]
89 ['nl_t1_clique_si_confidence', 1]
90 ['nl_t2_single_si', 3]
91 ['nl_t2_si

0 ['m_t2_clique_an', 3]
1 ['m_t2_clique_an_confidence', 1]
2 ['m_t2_single_jp', 4]
3 ['m_t2_single_jp_confidence', 4]
4 ['m_t3_clique_tl', 8]
5 ['m_t3_clique_tl_confidence', 4]
6 ['nl_t3_single_tl', 2]
7 ['nl_t3_single_tl_confidence', 5]
8 ['m_t3_clique_si', 7]
9 ['m_t3_clique_si_confidence', 2]
10 ['m_t3_clique_jp', 7]
11 ['m_t3_clique_jp_confidence', 2]
12 ['nl_t3_single_an', 2]
13 ['nl_t3_single_an_confidence', 4]
14 ['nl_t1_clique_tl', 3]
15 ['nl_t1_clique_tl_confidence', 3]
16 ['m_t3_single_an', 4]
17 ['m_t3_single_an_confidence', 3]
18 ['m_t1_clique_si', 3]
19 ['m_t1_clique_si_confidence', 2]
20 ['m_t2_clique_tl', 10]
21 ['m_t2_clique_tl_confidence', 3]
22 ['m_t2_single_an', 5]
23 ['m_t2_single_an_confidence', 3]
24 ['m_t2_single_an', 5]
25 ['m_t2_single_an_confidence', 3]
26 ['control1', 'yellow']
27 ['control1_confidence', 2]
28 ['m_t3_clique_an', 6]
29 ['m_t3_clique_an_confidence', 2]
30 ['m_t3_single_tl', 4]
31 ['m_t3_single_tl_confidence', 4]
32 ['nl_t2_clique_an', 1]
33 ['n

46 ['nl_t1_single_jp', 2]
47 ['nl_t1_single_jp_confidence', 5]
48 ['nl_t2_clique_jp', 3]
49 ['nl_t2_clique_jp_confidence', 4]
50 ['nl_t2_clique_jp', 3]
51 ['nl_t2_clique_jp_confidence', 4]
52 ['control2', 4]
53 ['control2_confidence', 5]
54 ['m_t2_single_si', 4]
55 ['m_t2_single_si_confidence', 5]
56 ['nl_t2_single_tl', 5]
57 ['nl_t2_single_tl_confidence', 4]
58 ['nl_t2_clique_tl', 2]
59 ['nl_t2_clique_tl_confidence', 3]
60 ['m_t1_single_si', 2]
61 ['m_t1_single_si_confidence', 5]
62 ['nl_t1_clique_jp', 2]
63 ['nl_t1_clique_jp_confidence', 5]
64 ['nl_t2_clique_an', 6]
65 ['nl_t2_clique_an_confidence', 5]
66 ['nl_t3_single_an', 2]
67 ['nl_t3_single_an_confidence', 5]
68 ['m_t3_clique_tl', 4]
69 ['m_t3_clique_tl_confidence', 4]
70 ['m_t1_single_tl', 3]
71 ['m_t1_single_tl_confidence', 5]
72 ['m_t1_single_jp', 3]
73 ['m_t1_single_jp_confidence', 5]
74 ['m_t2_clique_si', 8]
75 ['m_t2_clique_si_confidence', 3]
76 ['m_t2_single_an', 4]
77 ['m_t2_single_an_confidence', 5]
78 ['m_t2_single_an'

0 ['m_t3_clique_tl', 6]
1 ['m_t3_clique_tl_confidence', 1]
2 ['nl_t2_single_si', 2]
3 ['nl_t2_single_si_confidence', 3]
4 ['m_t1_single_tl', 3]
5 ['m_t1_single_tl_confidence', 5]
6 ['m_t2_single_si', 4]
7 ['m_t2_single_si_confidence', 4]
8 ['nl_t1_single_jp', 2]
9 ['nl_t1_single_jp_confidence', 5]
10 ['nl_t3_single_tl', 2]
11 ['nl_t3_single_tl_confidence', 5]
12 ['m_t1_single_an', 2]
13 ['m_t1_single_an_confidence', 5]
14 ['m_t2_single_an', 4]
15 ['m_t2_single_an_confidence', 3]
16 ['m_t3_single_an', 3]
17 ['m_t3_single_an_confidence', 3]
18 ['m_t1_clique_si', 1]
19 ['m_t1_clique_si_confidence', 4]
20 ['nl_t3_clique_jp', 5]
21 ['nl_t3_clique_jp_confidence', 1]
22 ['m_t1_clique_an', 2]
23 ['m_t1_clique_an_confidence', 4]
24 ['m_t1_clique_an', 2]
25 ['m_t1_clique_an_confidence', 4]
26 ['control1', 'red']
27 ['control1_confidence', 1]
28 ['nl_t2_single_tl', 4]
29 ['nl_t2_single_tl_confidence', 4]
30 ['nl_t2_clique_jp', 1]
31 ['nl_t2_clique_jp_confidence', 1]
32 ['m_t3_single_jp', 1]
33 ['